In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9532k      0  0:00:08  0:00:08 --:--:-- 11.3M


In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
 os.makedirs(val_dir / category)
 files = os.listdir(train_dir / category)
 random.Random(1337).shuffle(files) # Shuffle the list of training files using a seed, to ensure we get the same validation set every time we run the code.
 num_val_samples = int(0.2 * len(files))
 val_files = files[-num_val_samples:]
 for fname in val_files:
    shutil.move(train_dir / category / fname,val_dir / category / fname)

In [ ]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
 "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
 "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
 "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'I think a great many viewers missed entirely the fact that this is obviously a parody of western films.<br /><br />This is not a bad movie - it is a clever tongue in cheek take on westerns. I don\'t believe this film was taking itself seriously for a moment.<br /><br />What makes this film even more unique is the fact it is centered around 4 strong, beautiful women, two of which are black, one Asian, and a Mexican/Hispanic character.<br /><br />These aren\'t your usual western women--they\'re tough--they can draw fast and shoot straight.<br /><br />They\'re so tough even the bartender is shaking when he pours their whiskey.<br /><br />The plot which moves this story along is typical of westerns--in the vein of "you shot my brother--so I\'m gonna get you!" Only in this western, a woman\'s sister has been shot and she\'s out for vengeance on the gang who did it.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
# Limit the vocabulary to the 20,000 most frequent words. Otherwise we’d be indexing every word in the training data— potentially tens of thousands of terms that only occur once or
# twice and thus aren’t informative. In general, 20,000 is the right vocabulary size for text classification.

text_vectorization = layers.TextVectorization(
 max_tokens=20000,
 output_mode="multi_hot", # Encode the output tokens as multi-hot binary vectors.
)

In [ ]:
text_only_train_ds = train_ds.map(lambda x, y: x) # Prepare a dataset that only yields raw text inputs (no labels).

In [ ]:
text_vectorization.adapt(text_only_train_ds)

In [ ]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
 max_tokens=max_tokens,
 output_mode="int",
 output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

In [ ]:
int_train_ds = train_ds.map(
 lambda x, y: (text_vectorization(x), y),
 num_parallel_calls=4)
int_val_ds = val_ds.map(
 lambda x, y: (text_vectorization(x), y),
 num_parallel_calls=4)
int_test_ds = test_ds.map(
 lambda x, y: (text_vectorization(x), y),
 num_parallel_calls=4)

#  Understanding self-attention

As you’re going through this book, you may be skimming some parts and attentively
reading others, depending on what your goals or interests are.

What if your models
did the same? It’s a simple yet powerful idea: not all input information seen by a
model is equally important to the task at hand, so models should “pay more attention”
to some features and “pay less attention” to other features.

Does that sound familiar? You’ve already encountered a similar concept twice in
this book:


 Max pooling in convnets looks at a pool of features in a spatial region and
selects just one feature to keep. That’s an “all or nothing” form of attention:
keep the most important feature and discard the rest.


 TF-IDF normalization assigns importance scores to tokens based on how much
information different tokens are likely to carry. Important tokens get boosted
while irrelevant tokens get faded out. That’s a continuous form of attention.

There are many different forms of attention you could imagine, but they all start by computing importance scores for a set of features, with higher scores for more relevant features and lower scores for less relevant ones.

 How these scores should be
computed, and what you should do with them, will vary from approach to approach.

Crucially, this kind of attention mechanism can be used for more than just highlighting
or erasing certain features. It can be used to make features context-aware. You’ve just
learned about word embeddings—vector spaces that capture the “shape” of the semantic
relationships between different words. In an embedding space, a single word has a fixed
position—a fixed set of relationships with every other word in the space

But that’s not
quite how language works: the meaning of a word is usually context-specific. When you
mark the date, you’re not talking about the same “date” as when you go on a date, nor is
it the kind of date you’d buy at the market. When you say, “I’ll see you soon,” the meaning of the word “see” is subtly different from the “see” in “I’ll see this project to its end” or
“I see what you mean.” And, of course, the meaning of pronouns like “he,” “it,” “in,” etc.,
is entirely sentence-specific and can even change multiple times within a single sentence.

Clearly, a smart embedding space would provide a different vector representation
for a word depending on the other words surrounding it. That’s where self-attention
comes in. **The purpose of self-attention is to modulate the representation of a token**
**by using the representations of related tokens in the sequence**. This produces contextaware token representations

[How self attention works and how attention scores are calculated](https://chat.openai.com/share/b2b4096d-5756-4084-9b55-ef28c9d52a51)

In [ ]:
def self_attention(input_sequence):
 output = np.zeros(shape=input_sequence.shape)
 for i, pivot_vector in enumerate(input_sequence): # Iterate over each token in the input sequence
    scores = np.zeros(shape=(len(input_sequence),))
    for j, vector in enumerate(input_sequence):
      scores[j] = np.dot(pivot_vector, vector.T) # Compute the dot product (attention score) between the token and every other token.
    # Scale by a normalization factor, and apply a softmax.
    scores /= np.sqrt(input_sequence.shape[1])
    scores = softmax(scores)
    new_pivot_representation = np.zeros(shape=pivot_vector.shape)
    for j, vector in enumerate(input_sequence):
      new_pivot_representation += vector * scores[j] # Take the sum of all tokens weighted by the attention scores
    output[i] = new_pivot_representation # That sum is our output.
 return output

[Understanding self Attention and Multi Head Self Attention](https://chat.openai.com/share/cac3cb92-fcf4-4c5c-801a-bc609ec4689e)

The original Transformer architecture consists of two parts: a Transformer encoder that
processes the source sequence, and a Transformer decoder that uses the source sequence
to generate a translated version. You’ll learn about about the decoder part in a minute.


 Crucially, the encoder part can be used for text classification—it’s a very generic
module that ingests a sequence and learns to turn it into a more useful representation. Let’s implement a Transformer encoder and try it on the movie review sentiment
classification task.

#  Transformer encoder implemented as a subclassed Layer

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim # Size of the input token vectors
    self.dense_dim = dense_dim # Size of the inner dense layer
    self.num_heads = num_heads # Number of attention heads
    self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation="relu"),
        layers.Dense(embed_dim),]
        )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
  def call(self, inputs, mask=None): # Computation goes in call().
    if mask is not None: # The mask that will be generated by the Embedding layer will be 2D, but the attention layer expects to be 3D or 4D, so we expand its rank.
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
    inputs, inputs, attention_mask=mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)
  def get_config(self): # Implement serialization so we can save the model.
    config = super().get_config()
    config.update({"embed_dim": self.embed_dim,"num_heads": self.num_heads,"dense_dim": self.dense_dim,})
    return config

You’ll note that the normalization layers we’re using here aren’t BatchNormalization
layers like those we’ve used before in image models. That’s because BatchNormalization
doesn’t work well for sequence data. Instead, we’re using the LayerNormalization layer,
which normalizes each sequence independently from other sequences in the batch.
Like this, in NumPy-like pseudocode:

In [ ]:
def layer_normalization(batch_of_sequences): # Input shape: (batch_size, sequence_length, embedding_dim)

 # To compute mean and variance, we only pool data over the last axis (axis -1).
 mean = np.mean(batch_of_sequences, keepdims=True, axis=-1)
 variance = np.var(batch_of_sequences, keepdims=True, axis=-1)
 return (batch_of_sequences - mean) / variance

Compare to BatchNormalization (during training):

In [ ]:
def batch_normalization(batch_of_images): # Input shape: (batch_size, height, width, channels)

 # Pool data over the batch axis (axis 0), which creates interactions between samples in a batch.
 mean = np.mean(batch_of_images, keepdims=True, axis=(0, 1, 2))
 variance = np.var(batch_of_images, keepdims=True, axis=(0, 1, 2))
 return (batch_of_images - mean) / variance

While BatchNormalization collects information from many samples to obtain accurate statistics for the feature means and variances, LayerNormalization pools data
within each sequence separately, which is more appropriate for sequence data.

 Now that we’ve implemented our TransformerEncoder, we can use it to assemble a
text-classification model similar to the GRU-based one you’ve seen previously.

In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x) # Since TransformerEncoder returns full sequences, we need to reduce each sequence to a single vector for classification via a global pooling layer.
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

Since the TransformerEncoder returns a sequence, the GlobalMaxPooling1D layer is used to reduce each sequence to a single vector. This is a common technique to convert variable-length sequences into fixed-size representations that can be used for classification.

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Trans  (None, None, 256)         543776    
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [ ]:
callbacks = [
 keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
 save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
 callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 59s 87ms/step - loss: 0.5003 - accuracy: 0.7618 - val_loss: 0.4032 - val_accuracy: 0.8134
Epoch 2/20
625/625 [==============================] - 48s 77ms/step - loss: 0.3477 - accuracy: 0.8497 - val_loss: 0.3155 - val_accuracy: 0.8666
Epoch 3/20
625/625 [==============================] - 44s 70ms/step - loss: 0.3128 - accuracy: 0.8669 - val_loss: 0.3161 - val_accuracy: 0.8626
Epoch 4/20
625/625 [==============================] - 43s 68ms/step - loss: 0.2853 - accuracy: 0.8789 - val_loss: 0.3221 - val_accuracy: 0.8606
Epoch 5/20
625/625 [==============================] - 43s 68ms/step - loss: 0.2626 - accuracy: 0.8922 - val_loss: 0.2889 - val_accuracy: 0.8756
Epoch 6/20
625/625 [==============================] - 42s 67ms/step - loss: 0.2374 - accuracy: 0.9038 - val_loss: 0.3124 - val_accuracy: 0.8718
Epoch 7/20
625/625 [==============================] - 43s 70ms/step - loss: 0.2086 - accuracy: 0.9176 - val_loss: 0.2858 - val_accuracy:

In [ ]:
model = keras.models.load_model(
 "transformer_encoder.keras",
 custom_objects={"TransformerEncoder": TransformerEncoder}) # Provide the custom TransformerEncode  class to the model-loading process.
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 19s 23ms/step - loss: 0.3074 - accuracy: 0.8735
Test acc: 0.874


Transformer was a hybrid approach that is technically order-agnostic, but that manually
injects order information in the representations it processes. This is the missing ingredient! It’s called positional encoding

# USING POSITIONAL ENCODING TO RE-INJECT ORDER INFORMATION

The idea behind positional encoding is very simple: to give the model access to wordorder information, we’re going to add the word’s position in the sentence to each word
embedding. Our input word embeddings will have two components: the usual word vector, which represents the word independently of any specific context, and a position vector, which represents the position of the word in the current sentence. Hopefully, the model will then figure out how to best leverage this additional information.

The original “Attention is all you need” paper used an interesting trick to encode
word positions: it added to the word embeddings a vector containing values in the
range [-1, 1] that varied cyclically depending on the position (it used cosine functions to achieve this). This trick offers a way to uniquely characterize any integer in a
large range via a vector of small values. It’s clever, but it’s not what we’re going to use
in our case. We’ll do something simpler and more effective: we’ll learn positionembedding vectors the same way we learn to embed word indices. We’ll then proceed
to add our position embeddings to the corresponding word embeddings, to obtain a
position-aware word embedding. This technique is called “positional embedding.”

In [ ]:
class PositionalEmbedding(layers.Layer):
 def __init__(self, sequence_length, input_dim, output_dim, **kwargs): # A downside of position embeddings is that the sequence length needs to be known in advance
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim) # Prepare an Embedding layer for the token indices.
    self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim) # And another one for the token positions
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

 def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1) # delta=1: The step size between consecutive values in the sequence is 1.
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions # Add both embedding vectors together.

 def compute_mask(self, inputs, mask=None):
    # Like the Embedding layer, this layer should be able to generate a mask so we can ignore padding 0s in the inputs. The compute_mask method will called automatically by the framework, and the mask will get propagated
    # to the next layer.
    return tf.math.not_equal(inputs, 0)

 def get_config(self):
    # Implement serialization so we can save the model.
    config = super().get_config()
    config.update({
      "output_dim": self.output_dim,
      "sequence_length": self.sequence_length,
      "input_dim": self.input_dim,
    })
    return config

You would use this PositionEmbedding layer just like a regular Embedding layer

# PUTTING IT ALL TOGETHER: A TEXT-CLASSIFICATION TRANSFORMER

All you have to do to start taking word order into account is swap the old Embedding
layer with our position-aware version

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
 loss="binary_crossentropy",
 metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         5273600   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

In [ ]:
callbacks = [
 keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
 save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 61s 93ms/step - loss: 0.5391 - accuracy: 0.7395 - val_loss: 0.3223 - val_accuracy: 0.8532
Epoch 2/20
625/625 [==============================] - 51s 81ms/step - loss: 0.2979 - accuracy: 0.8748 - val_loss: 0.3500 - val_accuracy: 0.8718
Epoch 3/20
625/625 [==============================] - 46s 74ms/step - loss: 0.2384 - accuracy: 0.9050 - val_loss: 0.2759 - val_accuracy: 0.8886
Epoch 4/20
625/625 [==============================] - 46s 73ms/step - loss: 0.1967 - accuracy: 0.9232 - val_loss: 0.3200 - val_accuracy: 0.8790
Epoch 5/20
625/625 [==============================] - 45s 72ms/step - loss: 0.1647 - accuracy: 0.9359 - val_loss: 0.4637 - val_accuracy: 0.8686
Epoch 6/20
625/625 [==============================] - 45s 72ms/step - loss: 0.1415 - accuracy: 0.9459 - val_loss: 0.4335 - val_accuracy: 0.8796
Epoch 7/20
625/625 [==============================] - 45s 73ms/step - loss: 0.1199 - accuracy: 0.9559 - val_loss: 0.5659 - val_accuracy:

In [ ]:
model = keras.models.load_model(
 "full_transformer_encoder.keras",
 custom_objects={"TransformerEncoder": TransformerEncoder,
 "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 22s 28ms/step - loss: 0.2905 - accuracy: 0.8792
Test acc: 0.879


We get to 88.3% test accuracy, a solid improvement that clearly demonstrates the
value of word order information for text classification. This is our best sequence
model so far! However, it’s still one notch below the bag-of-words approach.

#  When to use sequence models over bag-of-words models

You may sometimes hear that bag-of-words methods are outdated, and that Transformerbased sequence models are the way to go, no matter what task or dataset you’re looking at. This is definitely not the case: a small stack of Dense layers on top of a bag-ofbigrams remains a perfectly valid and relevant approach in many cases. In fact, among
the various techniques that we’ve tried on the IMDB dataset throughout this chapter,
the best performing so far was the bag-of-bigrams!

 In 2017, my team and I ran a systematic analysis of the performance of various textclassification techniques across many different types of text datasets, and we discovered a remarkable and surprising rule of thumb for deciding whether to go with a
bag-of-words model or a sequence model (http://mng.bz/AOzK)     —a golden constant
of sorts.   

It turns out that when approaching a new text-classification task, you should pay
close attention to the ratio between the number of samples in your training data and
the mean number of words per sample . If that ratio is small—less
than 1,500—then the bag-of-bigrams model will perform better (and as a bonus, it will
be much faster to train and to iterate on too). If that ratio is higher than 1,500, then
you should go with a sequence model. In other words, sequence models work best
when lots of training data is available and when each sample is relatively short.